# **Task: Using the Drone's front camera, detect an AR tag, identify the Tag ID, then compute the drone's pose from the AR tag's information**

Steps:

1. Use a program to extract the AR tag and convert to a matrix to find the TAG ID
2. Corrospond the AR tag to known location values in a pre-determined index
3. Using the points of the AR tag, output the drone's pose in a 6 dimensional vector

# Imports

In [2]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import cv2.aruco as aruco
import os
import glob
from matplotlib.patches import Rectangle
from scipy.stats import linregress
import matplotlib.patches as patches
from matplotlib.lines import Line2D

# Step 1

**Use a program to extract the AR tag to find the tag ID**

In [ ]:
folderpath = "ARTags/"
imagepaths = glob.glob(os.path.join(folderpath, "*.png"))
kernel = np.ones((3, 3),np.uint8)
print(cv2.__version__)

for path in imagepaths:
    tag = cv2.imread(path)

    
    gray = cv2.cvtColor(tag, cv2.COLOR_BGR2GRAY)

    tag = cv2.convertScaleAbs(gray, alpha=1.0, beta=-50)
    tag = cv2.erode(tag, kernel)


    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_5X5_100)

    detectorParams = aruco.DetectorParameters()
    detectorParams.aprilTagCriticalRad = 0.1 #default 0.17
    detectorParams.aprilTagMaxLineFitMse = 100 #default 10
    detectorParams.aprilTagMaxNmaxima = 15 #default 10
    detectorParams.polygonalApproxAccuracyRate = 0.01 #default 0.03
    detectorParams.useAruco3Detection = True
    detector = aruco.ArucoDetector(aruco_dict, detectorParams)
    corners, ids, rejected = detector.detectMarkers(gray)

    if ids is not None: 

        print(f"Detected ID: {ids.flatten()} on image: {path}")
        tag = aruco.drawDetectedMarkers(tag, corners, ids)
    else:
        print(f"No markers found for image: {path}")
        # fig, ax = plt.subplots()
        # ax.imshow(image, cmap='gray')

        plt.show()





4.12.0
<class 'numpy.ndarray'>
Detected ID: [87] on image: ARTags/id87.png
<class 'numpy.ndarray'>
Detected ID: [93] on image: ARTags/image5.png
<class 'numpy.ndarray'>
Detected ID: [95] on image: ARTags/id95.png
<class 'numpy.ndarray'>
Detected ID: [95] on image: ARTags/id95_2.png
<class 'numpy.ndarray'>
Detected ID: [95] on image: ARTags/image3.png
<class 'NoneType'>
No markers found for image: ARTags/image.png
<class 'numpy.ndarray'>
Detected ID: [93] on image: ARTags/image6.png
<class 'numpy.ndarray'>
Detected ID: [87] on image: ARTags/id87_2.png
<class 'numpy.ndarray'>
Detected ID: [93] on image: ARTags/id93_1.png
<class 'NoneType'>
No markers found for image: ARTags/id87_1.png


KeyboardInterrupt: 

# Step 2 
**Corrospond the AR tag to a predetermined location**

In [6]:
T = np.array([[0, 0, 1, 0],
            [0, -1, 0, 0],
            [1, 0, 0, 0],
            [0, 0, 0, 1]])

print(T[:3, :3])
print()
print(T[:3, 3].T)

[[ 0  0  1]
 [ 0 -1  0]
 [ 1  0  0]]

[0 0 0]


# Step 3

**Find the drone's position relative to the tag's position**

In [ ]:
MARKER_LENGTH = 0.2667 #Meters
CAMERA_MATRIX = np.array([])
DIST_COEFFS = []

def rotationMatrixToEulerAngles(R):

    sy = np.linalg.norm([R[0, 0], R[1, 0]])
    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])  
        y = np.arctan2(-R[2, 0], sy)      
        z = np.arctan2(R[1, 0], R[0, 0])  
    else:
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])


def func(corners):

    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, MARKER_LENGTH, CAMERA_MATRIX, DIST_COEFFS)

    R_marker_to_cam, _ = cv2.Rodrigues(rvecs)

    R_cam_to_marker = R_marker_to_cam.T

    t_cam_to_marker = -R_cam_to_marker @ tvecs #The position of the camera in the marker's coordinate frame

    rvec_cam_to_marker, _ = cv2.Rodrigues(R_cam_to_marker) #The rotation of the camera in relation to the marker (in marker frame)

    x, y, z = rotationMatrixToEulerAngles(R_cam_to_marker) #Roll, pitch, yaw
        


# Step 4

**Write the ros node**

In [ ]:

import rclpy
from rclpy.node import Node
import cv2
from std_msgs.msg import String
from std_msgs.msg import Int32
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError
import cv2.aruco as aruco
import sys

BASE_LOCATION = np.array([0, 0, 0])
CAR_TAG_ID = 96
CAR_MARKER_LENGTH = 0 #TBD -- Meters
MARKER_LENGTH = 0.2667 #Meters
CAMERA_MATRIX = np.array([])
DIST_COEFFS = ()
KERNEL = np.ones((5, 5),np.uint8)   
T_FC_2_BD= np.array([[0, 0, 1, 0],
            [0, -1, 0, 0],
            [1, 0, 0, 0],
            [0, 0, 0, 1]]) @ np.array(
            [[1, 0, 0, 0],
            [0, 0, 1, 0],
            [0, -1, 0, 0],
            [0, 0, 0, 1]])
T_MARKERS = []

class ARTagDetector(Node):
    def __init__(self):
        super().__init__('artag_detector')
        self.get_logger().info('ARTag Detector has been initialized')
        
        #Create camera subscription, 50 milli second wait
        self.camera_sub = self.create_subscription(
            Image,
            '/world/line_following_track/model/x500_mono_cam_down_0/link/camera_link/sensor/imager/image',
            self.camera_sub_cb,
            50
        )

        self.tagid_pub = self.create_publisher(String, '/', 1)
        self.position_pub = self.create_publisher(Int32, '/line/detector_image', 1)
        self.bridge = CvBridge()

    """
    camera callback
    Publishes to tag id and position pub
    
    """
    def camera_sub_cb(self, msg):
        # Convert Image msg to OpenCV image
        image = self.bridge.imgmsg_to_cv2(msg, "mono8")

        # Detect line in the image. detect returns a parameterize the line (if one exists)
        tagids, corners = self.detect_id(image)
        if tagids is None:
            self.get_logger().info('No ARTag found!')
        else:
            self.get_logger().info(f'ARTag Detected! Detected: {tagids.flatten()}')



        # Aruco Estimate pose
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, MARKER_LENGTH, CAMERA_MATRIX, DIST_COEFFS)

        worldPose = self.calculateDronePose(rvecs,tvecs,tagids)



    def calculateDronePose(self, rvecs, tvecs, tagids):
        #Rotate to Euler angles
        def rotationMatrixToEulerAngles(R):
            sy = np.linalg.norm([R[0, 0], R[1, 0]])
            singular = sy < 1e-6

            if not singular:
                x = np.arctan2(R[2, 1], R[2, 2])  
                y = np.arctan2(-R[2, 0], sy)      
                z = np.arctan2(R[1, 0], R[0, 0])  
            else:
                x = np.arctan2(-R[1, 2], R[1, 1])
                y = np.arctan2(-R[2, 0], sy)
                z = 0
            return np.array([x, y, z])

        posTags = {}
        
        # Processing each tag...
        for i in range(len(tagids)):
            T = np.eye(4)
            marker_id = tagids[i]
            T_marker = T_MARKERS[marker_id]

            rvec = rvecs[i]
            tvec = tvecs[i]

            #Turn Rotation to world frame
            R_marker_to_cam, _ = cv2.Rodrigues(rvec)
            R_cam_to_marker = R_marker_to_cam.T
            #The rotation of the camera in relation to the marker (in marker frame)
            rvec_cam_to_marker, _ = cv2.Rodrigues(R_cam_to_marker) 
            roll, pitch, yaw = rotationMatrixToEulerAngles(rvec_cam_to_marker)     #Roll, pitch, yaw

            #Turn translation to world frame
            T[:3, :3] = R_marker_to_cam
            T[:3, 3] = tvec.flatten()
            T_fc_in_marker = np.linalg.inv(T) 
            T_fc_in_lenu = T_marker @ T_fc_in_marker 
            #The position of the camera in the marker's coordinate frame (NOT RELATIVE TO THE MARKER)
            T_bd_in_lenu = T_fc_in_lenu @ T_FC_2_BD

            posTags[marker_id] = T_bd_in_lenu[:3, 3].T 
        
        

    
    """
    Function Description: Passing in a grayscale cv2 image and detects the AR tag based on the aruco 5x5 dictionary
    with numbers 1-100 (WARNING: IF TAG NOT FULLY VISIBLE AS SQUARE WILL NOT WORK)

    Input: Cv2 Image
    Output: numpy array of TAGIDs (may be multiple)
    """
    def detect_id(self, image):
        image = cv2.convertScaleAbs(gray, alpha=1.0, beta=-50)
        image = cv2.erode(image, kernel)


        aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_5X5_100)

        detectorParams = aruco.DetectorParameters()
        detectorParams.aprilTagCriticalRad = 0.1 #default 0.17
        detectorParams.aprilTagMaxLineFitMse = 100 #default 10
        detectorParams.aprilTagMaxNmaxima = 15 #default 10
        detectorParams.polygonalApproxAccuracyRate = 0.01 #default 0.03
        detectorParams.useAruco3Detection = True
        detector = aruco.ArucoDetector(aruco_dict, detectorParams)
        corners, ids, rejected = detector.detectMarkers(gray)

        return ids.flatten(), corners

def main(args=None):
    rclpy.init(args=args)
    detector = ARTagDetector()
    detector.get_logger.info('ARTag detector initialized')
    try:
        rclpy.spin(detector)
    except KeyboardInterrupt:
        print("ARTag detector shutting down")
    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        detector.destroy_node()
        rclpy.shutdown()


if __name__=='__main__':#Unneeded if you only ever run the node directly with ros2 run 
    main()



Making separate ros node for ar tag for car...

In [ ]:

from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import cv2.aruco as aruco
import os
import glob
from matplotlib.patches import Rectangle
from scipy.stats import linregress
import matplotlib.patches as patches
from matplotlib.lines import Line2D
import math

IDS = []
CURR_DIST = [0.0, 0.0, 0.0]
MARKER_SIZE = 26.6 #Centimeters
ARUCODICT = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_100)
IN_MATRIX = np.array([[1.29650191e+03, 0.00000000e+00, 4.14079631e+02],#intrinsic camera matrix, calibrate with
                    [0.00000000e+00, 1.29687850e+03, 2.26798449e+02],#https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
                    [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
DISTORTION = np.array([[2.56660190e-01, -2.23374068e+00, -2.54856563e-03,  4.09905103e-03,9.68094572e+00]])
MARKER_POINTS = np.array([[-MARKER_SIZE / 2, MARKER_SIZE / 2, 0],#corners of AR in world frame
                [MARKER_SIZE / 2, MARKER_SIZE / 2, 0],
                [MARKER_SIZE / 2, -MARKER_SIZE / 2, 0],
                [-MARKER_SIZE / 2, -MARKER_SIZE / 2, 0]], dtype=np.float32)

    
def rodrigues_to_euler(rvec):
    # Convert Rodrigues vector to rotation matrix
    rotation_matrix, _ = cv2.Rodrigues(rvec)
    # Check for gimbal lock
    sy = np.sqrt(rotation_matrix[0, 0] ** 2 + rotation_matrix[1, 0] ** 2)
    locked = sy < 1e-6

    if not locked:
        roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
        pitch = np.arctan2(-rotation_matrix[2, 0], sy)
        yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    else:
        roll = np.arctan2(-rotation_matrix[1, 2], rotation_matrix[1, 1])
        pitch = np.arctan2(-rotation_matrix[2, 0], sy)
        yaw = 0

    return np.array([roll, pitch, yaw])

        
def grab_tags(tag):
    global IDS
    # Preprocess
    if tag is None:
        print("Error: Image is None.")
        return []
    tag = cv2.cvtColor(tag, cv2.COLOR_BGR2GRAY)
    
    tag = cv2.convertScaleAbs(gray, alpha=1.0, beta=-50)
    tag = cv2.erode(tag, kernel)


    detectorParams = aruco.DetectorParameters()
    detectorParams.aprilTagCriticalRad = 0.1 #default 0.17
    detectorParams.aprilTagMaxLineFitMse = 100 #default 10
    detectorParams.aprilTagMaxNmaxima = 15 #default 10
    detectorParams.polygonalApproxAccuracyRate = 0.01 #default 0.03
    detectorParams.useAruco3Detection = True
    detector = aruco.ArucoDetector(ARUCODICT, detectorParams)
    corners, ids, rejected = detector.detectMarkers(gray)
    IDS = ids

    # rvec, tvec = my_estimatePoseSingleMarkers2(corners, mtx, distortion)#get stats from AR tag
    #rvec=rodrigues vector, tvec=translation vector of the AR tag in the camera frame
    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, MARKER_SIZE, IN_MATRIX, DISTORTION)

    # cv.drawFrameAxes(tag, mtx, distortion, rvec, tvec, length=10)#optional, helps visualization
    #red=x, y=green, z=blue
    # print(f"rvecs: {rvec}, tvecs: {tvec}\n")
    return rvecs, tvecs

def find_pos(rvec1, tvec1):
    rot_mat, jacobian = cv2.Rodrigues(rvec1)
    wRd = np.transpose(rot_mat)#turns drone frame to world frame
    drone_from_ar = np.matmul(wRd, -tvec1)#finds translation vector from tag to drone in world frame
    #negative because tvec1 is from camera to tag, we want from tag to camera
    orien = rodrigues_to_euler(rvec1)
    return drone_from_ar, orien



def find_relative_pose(pic):
    #Each AR tag has a number written on it and a unique color combo, this will contain info for each 
    result = grab_tags(pic, ARUCODICT, IN_MATRIX, DISTORTION)#find and process the image
    maxDist = 0
    maxTrans,  maxOrien, id = None, None, None
    for index, res in enumerate(result):
        rvec, tvec = res
        tvec_m = tvec*0.01#convert to meters, the 26.6 market size was in cm
        trans, orien = find_pos(rvec, tvec_m)#get position relative to AR tag
        d = math.dist(trans, CURR_DIST)
        if d < maxDist:
            maxTrans = trans
            maxDist = d
            maxOrien = orien
            id = IDS[index]
    
    return maxTrans, maxOrien, 
        

path = "PICAM_ARTAGS/"
image = cv2.imread(path, cv2.IMREAD_COLOR)
if image is None:
    print("Failed to load image. Check the path.")

trans, orien = find_relative_pose(image)
print(f"Relative distance from tag: {trans} and orientation according to tag: {orien}")
cv2.imshow("Pose Debug", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Making a camera feed processing


In [ ]:
import cv2
import numpy as np
import os

LOW = np.array([250, 250, 250])  # Lower image thresholding bound
HI = np.array([255, 255, 255])   # Upper image thresholding bound

KERNEL_D = np.ones((30, 30), np.uint8)
KERNEL_E = np.ones((20, 20), np.uint8)

filename = "easdifj"

vidpath = f"cameravids/{filename}.h64"
outpath = f"cameravids/{filename}_processed.mp4"
interval = 0.1

cap = cv2.VideoCapture(vidpath)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps * interval)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can also use 'XVID'
out = cv2.VideoWriter(outpath, fourcc, fps, (width, height))

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Only process every nth frame
    if frame_count % frame_interval == 0:
        # (Optional) Do processing here
        image = frame
        image = cv2.dilate(image, KERNEL_D, iterations = 1)
        image = cv2.erode(image, KERNEL_E, iterations = 1)
        mask = cv2.inRange(image, LOW, HI)
        image = cv2.bitwise_and(image, image, mask=mask)
        image = cv2.GaussianBlur(image, (5,5), 0)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, image = cv2.threshold(image,245,255,cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnt_sort = lambda cnt: (max(cv2.minAreaRect(cnt)[1]))
        sorted_contours = sorted(contours, key=cnt_sort, reverse=True)

        if len(sorted_contours) == 0:
            cv2.putText(image, "No line detected!", (image.shape[0]//2, image.shape[1]//2), color=(0, 0, 255), thickness=2)

        else:
            all_points = np.vstack(sorted_contours[0])
            [vx, vy, x, y] = cv2.fitLine(all_points, cv2.DIST_L2, 0, 0.01, 0.01)
            x, y, vx, vy = x.item(), y.item(), vx.item(), vy.item()

            # rows, cols = image.shape
            left_y = int((-x * vy / vx) + y)
            right_y = int(((cols - x) * vy / vx) + y)
            cv2.line(image, (0, left_y), (image.shape[1] - 1, right_y), (0, 255, 0), 2)
            
        out.write(image)  # Save to output video

    frame_count += 1

cap.release()
out.release()
print(f"Success! Video saved")



